## SynDiffix Usage Tutorial

This notebook demonstrates how to use SynDiffix for a loan default prediction model. It is based on the example by Zhou Xu at:

https://github.com/zhouxu-ds/loan-default-prediction/blob/main/notebook/modeling.ipynb

which builds a model to predict the likelihood of a loan default using the Czech banking dataset.

### Setup

The `syndiffix` package requires Python 3.10 or later. Let's install it and other packages we'll need for the notebook.

In [121]:
%pip install -q syndiffix requests pandas matplotlib numpy seaborn scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Loading the datasets

These table have all been prepared and loaded onto the open-diffix.org website.

In [1]:
import requests
import bz2
import pickle
def download_and_load(url):
    response = requests.get(url)
    data = bz2.decompress(response.content)
    df = pickle.loads(data)
    return df

# Usage
df_loan = download_and_load('http://open-diffix.org/datasets/loan.pbz2')
df_account = download_and_load('http://open-diffix.org/datasets/account.pbz2')
df_district = download_and_load('http://open-diffix.org/datasets/district.pbz2')
df_order = download_and_load('http://open-diffix.org/datasets/order.pbz2')
df_trans = download_and_load('http://open-diffix.org/datasets/trans.pbz2')
df_disp = download_and_load('http://open-diffix.org/datasets/disp.pbz2')
df_card = download_and_load('http://open-diffix.org/datasets/card.pbz2')
df_client = download_and_load('http://open-diffix.org/datasets/client.pbz2')
df_client.rename(columns={'district_id': 'cli_district_id'}, inplace=True)
df_card.rename(columns={'type': 'card_type'}, inplace=True)
df_account.rename(columns={'date': 'acct_date'}, inplace=True)

### Extract feature

Zhou extracted a specific set of features that he used for his model. We copy that here.

In [2]:

import pandas as pd

df_loan_acct = pd.merge(df_loan, df_account, on='account_id', how='left')
df = pd.merge(df_loan_acct, df_district, on='district_id', how='left')
df['days_between'] = (df['loan_date'] - df['acct_date']).dt.days
print(df.columns)
print(f"There are {len(df)} loans")

Index(['loan_id', 'account_id', 'loan_date', 'amount', 'duration', 'payments',
       'status', 'defaulted', 'district_id', 'frequency', 'acct_date', 'city',
       'region', 'population', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
       'avg_salary', 'A12', 'A13', 'entrepreneur_rate', 'A15', 'A16',
       'average_unemployment_rate', 'average_crime_rate', 'days_between'],
      dtype='object')
There are 682 loans


In [3]:
df_order_grouped = df_order.groupby('account_id')['amount'].mean().reset_index()
df_order_grouped.rename(columns={'amount': 'avg_order_amount'}, inplace=True)
df = pd.merge(df, df_order_grouped, on='account_id', how='left')
print(df.columns)
print(f"There are {len(df)} loans")

Index(['loan_id', 'account_id', 'loan_date', 'amount', 'duration', 'payments',
       'status', 'defaulted', 'district_id', 'frequency', 'acct_date', 'city',
       'region', 'population', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
       'avg_salary', 'A12', 'A13', 'entrepreneur_rate', 'A15', 'A16',
       'average_unemployment_rate', 'average_crime_rate', 'days_between',
       'avg_order_amount'],
      dtype='object')
There are 682 loans


In [4]:
df_avg_bal = df_trans.groupby('account_id')['balance'].mean().reset_index()
df_avg_bal.rename(columns={'balance': 'avg_trans_balance'}, inplace=True)
df_avg_amt = df_trans.groupby('account_id')['amount'].mean().reset_index()
df_avg_amt.rename(columns={'amount': 'avg_trans_amount'}, inplace=True)
df_cnt = df_trans.groupby('account_id').count().iloc[:, 1]
df_cnt.name = 'n_trans'
df = pd.merge(df, df_avg_bal, on='account_id', how='left')
df = pd.merge(df, df_avg_amt, on='account_id', how='left')
df = pd.merge(df, df_cnt, on='account_id', how='left')
print(df.columns)
print(len(df))

Index(['loan_id', 'account_id', 'loan_date', 'amount', 'duration', 'payments',
       'status', 'defaulted', 'district_id', 'frequency', 'acct_date', 'city',
       'region', 'population', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
       'avg_salary', 'A12', 'A13', 'entrepreneur_rate', 'A15', 'A16',
       'average_unemployment_rate', 'average_crime_rate', 'days_between',
       'avg_order_amount', 'avg_trans_balance', 'avg_trans_amount', 'n_trans'],
      dtype='object')
682


In [5]:
df_disp_owners = df_disp[df_disp['type'] == 'OWNER']
df = pd.merge(df, df_disp_owners, on='account_id', how='left')
df = pd.merge(df, df_card, on='disp_id', how='left')
df['card_type'].fillna('No', inplace=True)
print(df.groupby('card_type').count().iloc[:, 1])
print(df.columns)
print(len(df))

card_type
No         512
classic    133
gold        16
junior      21
Name: account_id, dtype: int64
Index(['loan_id', 'account_id', 'loan_date', 'amount', 'duration', 'payments',
       'status', 'defaulted', 'district_id', 'frequency', 'acct_date', 'city',
       'region', 'population', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
       'avg_salary', 'A12', 'A13', 'entrepreneur_rate', 'A15', 'A16',
       'average_unemployment_rate', 'average_crime_rate', 'days_between',
       'avg_order_amount', 'avg_trans_balance', 'avg_trans_amount', 'n_trans',
       'disp_id', 'client_id', 'type', 'card_id', 'card_type', 'issued'],
      dtype='object')
682


In [6]:
df = pd.merge(df, df_client, on='client_id', how='left')
df['same_district'] = df['district_id'] == df['cli_district_id']
df['owner_age'] = (df['loan_date'] - df['birth_number']).dt.days // 365
print(df.columns)
print(len(df))

Index(['loan_id', 'account_id', 'loan_date', 'amount', 'duration', 'payments',
       'status', 'defaulted', 'district_id', 'frequency', 'acct_date', 'city',
       'region', 'population', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
       'avg_salary', 'A12', 'A13', 'entrepreneur_rate', 'A15', 'A16',
       'average_unemployment_rate', 'average_crime_rate', 'days_between',
       'avg_order_amount', 'avg_trans_balance', 'avg_trans_amount', 'n_trans',
       'disp_id', 'client_id', 'type', 'card_id', 'card_type', 'issued',
       'birth_number', 'cli_district_id', 'sex', 'same_district', 'owner_age'],
      dtype='object')
682


Here is the final feature list selected by Zhou.

In [7]:
df_ml = df[['amount', 'duration', 'payments', 'days_between', 'population', 
            'avg_salary', 'average_unemployment_rate', 'entrepreneur_rate', 
            'average_crime_rate', 'avg_order_amount', 'avg_trans_amount',
            'avg_trans_balance', 'n_trans', 'owner_age', 
            'frequency', 'card_type', 'same_district', 'sex', 'defaulted']]

TODO: Do the correlations by synthesizing each pair and taking the correlations. Then compare with the original

### Prepare data for synthesis

Before we synthesize the data, we need to split the original into training and test dataframes. This is because we will test the synthetic data model against the original test data, and so that data cannot be included in what gets synthesized.

In [39]:
df_train = df_ml.sample(n=int(len(df_ml)*0.7))
df_test = df_ml.drop(df_train.index)

### Synthesize the data

Since we know the target column, we should specify it when we synthesize the data. This will lead to better predictions. There are two options. One is to ask SynDiffix to synthesize every column, and the other is to ask SynDiffix to synthesize only those columns that it determines are good features. We do both here so that we may compare them.

In [40]:
from syndiffix import Synthesizer
from syndiffix.clustering.strategy import MlClustering

target_column = 'defaulted'
df_syn_all = Synthesizer(df_train, target_column=target_column).sample()

df_syn_feat = Synthesizer(df_train, clustering=MlClustering(target_column="defaulted", drop_non_features=True)).sample()
feat_cols = list(df_syn_feat.columns)
feat_cols.remove(target_column)

After this, we replicate all transformation and modeling opeations on both the original and synthesized data so that we can compare them.

In [41]:
print("Synthetic data columns without features:")
print(df_syn_feat.columns, feat_cols)
print("Original data columns:")
print(df_train.columns)
print(f"{len(df_syn_feat)} (features only) and {len(df_syn_all)} (all) synthetic data rows, and {len(df_train)} original data rows")

Synthetic data columns without features:
Index(['payments', 'avg_trans_amount', 'avg_trans_balance', 'defaulted'], dtype='object') ['payments', 'avg_trans_amount', 'avg_trans_balance']
Original data columns:
Index(['amount', 'duration', 'payments', 'days_between', 'population',
       'avg_salary', 'average_unemployment_rate', 'entrepreneur_rate',
       'average_crime_rate', 'avg_order_amount', 'avg_trans_amount',
       'avg_trans_balance', 'n_trans', 'owner_age', 'frequency', 'card_type',
       'same_district', 'sex', 'defaulted'],
      dtype='object')
476 (features only) and 476 (all) synthetic data rows, and 682 original data rows


From the above, we see that SynDiffix found only two columns to be important features.

### Transformations

In [38]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
# Original
num_cols_orig = df_ml.columns[:-5]
cat_cols_orig = df_ml.columns[-5:]
col_trans_orig = ColumnTransformer([
    ('num', MinMaxScaler(), num_cols_orig),
    ('cat', OneHotEncoder(drop='if_binary'), cat_cols_orig)
])
df_transformed_orig = col_trans_orig.fit_transform(df_ml)
X_orig = df_transformed_orig[:, :-1]
y_orig = df_transformed_orig[:, -1]

#Synthetic (all columns)
num_cols_syn_all = df_syn_all.columns[:-5]
cat_cols_syn_all = df_syn_all.columns[-5:]
col_trans_syn_all = ColumnTransformer([
    #('num', MinMaxScaler(), num_cols_syn_all),
    ('num', StandardScaler(), num_cols_syn_all),
    ('cat', OneHotEncoder(drop='if_binary'), cat_cols_syn_all)
])
df_transformed_syn_all = col_trans_syn_all.fit_transform(df_syn_all)
X_syn_all = df_transformed_syn_all[:, :-1]
y_syn_all = df_transformed_syn_all[:, -1]

# For now, we're not doing any transformation on the features-only synthetic data
X_syn_feat = df_syn_feat[feat_cols]
y_syn_feat = df_syn_feat[[target_column]]

Index(['avg_trans_amount', 'avg_trans_balance'], dtype='object')
Index(['defaulted'], dtype='object')


### Modeling



In [29]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

# Original
# Train test split
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X_orig, y_orig, test_size=0.3, stratify=y_orig, random_state=10)
# See the inital model performance
clf_orig = RandomForestClassifier(random_state=10)
acc_orig = cross_val_score(clf_orig, X_train_orig, y_train_orig, cv=StratifiedKFold(n_splits=5), scoring='accuracy').mean()
f1_orig = cross_val_score(clf_orig, X_train_orig, y_train_orig, cv=StratifiedKFold(n_splits=5), scoring='f1').mean()
roc_auc_orig = cross_val_score(clf_orig, X_train_orig, y_train_orig, cv=StratifiedKFold(n_splits=5), scoring='roc_auc').mean()

# Synthetic
# We want to test the synthetic models against the original data. We therefore strictly speaking don't need to split off test data here. Nevertheless, we do it to keep the comparison apples-to-apples.
X_train_syn_all, _, y_train_syn_all, _ = train_test_split(X_syn_all, y_syn_all, test_size=0.3, stratify=y_syn_all, random_state=10)
# See the inital model performance
clf_syn_all = RandomForestClassifier(random_state=10)
acc_syn_all = cross_val_score(clf_syn_all, X_train_syn_all, y_train_syn_all, cv=StratifiedKFold(n_splits=5), scoring='accuracy').mean()
f1_syn_all = cross_val_score(clf_syn_all, X_train_syn_all, y_train_syn_all, cv=StratifiedKFold(n_splits=5), scoring='f1').mean()
roc_auc_syn_all = cross_val_score(clf_syn_all, X_train_syn_all, y_train_syn_all, cv=StratifiedKFold(n_splits=5), scoring='roc_auc').mean()

print(f"Accuracy: Orig: {acc_orig}, Syn: {acc_syn_all}")
print(f"F1: Orig: {f1_orig}, Syn: {f1_syn_all}")
print(f"ROC AUC: Orig: {roc_auc_orig}, Syn: {roc_auc_syn_all}")

Accuracy: Orig: 0.8889473684210525, Syn: 0.8886622807017543
F1: Orig: 0.06666666666666668, Syn: 0.0
ROC AUC: Orig: 0.7124570919276801, Syn: 0.6939982174688056


In [ ]:
from syndiffix import Synthesizer
from syndiffix.clustering.strategy import MlClustering

df_syn = Synthesizer(df_ml, clustering=MlClustering(target_column="defaulted", drop_non_features=True)).sample()